In [1]:
import pandas as pd 
import pm4py
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#log = pm4py.read_xes("../data/InternationalDeclarations.xes.gz")
#df = pm4py.convert_to_dataframe(log)

## Load Data Preprocessed in ProM

In [3]:
log_time = pm4py.read_xes("../data/International_Declarations_final.xes")
df_time = pm4py.convert_to_dataframe(log_time)
#log_time = pm4py.read_xes('../data/International_Declarations_final.xes')

/Users/hannes/opt/anaconda3/envs/APM/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
parsing log, completed traces :: 100%|████| 5646/5646 [00:02<00:00, 2214.18it/s]


In [4]:
working_df = df_time
print(len(working_df.index))
working_df.head()

65376


,org:resource,org:role,remaining_time,id,concept:name,time:timestamp,case:Permit travel permit number,case:DeclarationNumber,case:Amount,case:RequestedAmount,...,case:id,case:Permit ID,case:Permit id,case:BudgetNumber,case:Permit ActivityNumber,case:AdjustedAmount,case:concept:name,trip_duration,remaining_time_since_end_trip,time_submission
0,STAFF MEMBER,EMPLOYEE,4577.514722,rv_travel permit 76455_6,Start trip,2016-10-04 22:00:00+00:00,travel permit number 76456,declaration number 76458,39.664561,39.664561,...,declaration 76457,travel permit 76455,travel permit 76455,budget 144133,activity 46005,39.664561,declaration 76457,NaN,NaN,NaN
1,STAFF MEMBER,EMPLOYEE,4577.514722,rv_travel permit 76455_7,End trip,2016-10-04 22:00:00+00:00,travel permit number 76456,declaration number 76458,39.664561,39.664561,...,declaration 76457,travel permit 76455,travel permit 76455,budget 144133,activity 46005,39.664561,declaration 76457,0.0,4577.514722,NaN
2,STAFF MEMBER,EMPLOYEE,171.978611,st_step 76459_0,Permit SUBMITTED by EMPLOYEE,2017-04-06 11:32:10+00:00,travel permit number 76456,declaration number 76458,39.664561,39.664561,...,declaration 76457,travel permit 76455,travel permit 76455,budget 144133,activity 46005,39.664561,declaration 76457,NaN,NaN,NaN
3,STAFF MEMBER,SUPERVISOR,171.973611,st_step 76460_0,Permit FINAL_APPROVED by SUPERVISOR,2017-04-06 11:32:28+00:00,travel permit number 76456,declaration number 76458,39.664561,39.664561,...,declaration 76457,travel permit 76455,travel permit 76455,budget 144133,activity 46005,39.664561,declaration 76457,NaN,NaN,NaN
4,STAFF MEMBER,EMPLOYEE,147.877500,st_step 76461_0,Declaration SUBMITTED by EMPLOYEE,2017-04-07 11:38:14+00:00,travel permit number 76456,declaration number 76458,39.664561,39.664561,...,declaration 76457,travel permit 76455,travel permit 76455,budget 144133,activity 46005,39.664561,declaration 76457,NaN,NaN,NaN


## Remove all data recorded after "End trip"
* We do not do prefix abstraction in this case, as we are specifically interested in predicting the remaining time after a specific event (End Trip) has happend
* Both for training and test data we thus remove this information
* The posterior feature (remaining time) was already computed in ProM for the "End trip" event

In [5]:
eventColumnName = "concept:name"
caseIdColumnName = "case:id"
caseIds = working_df[caseIdColumnName].unique()

index_before_End = []

for caseId in caseIds:
    cur_trace = working_df[working_df[caseIdColumnName] == caseId]
    end_trip_time = pd.Timestamp(cur_trace[cur_trace[eventColumnName] == "End trip"]["time:timestamp"].values[0])
    
    for i, event in cur_trace.iterrows():
        if pd.Timestamp(event["time:timestamp"]).replace(tzinfo=None) <= end_trip_time.tz_localize(None):
            index_before_End.append(i)
        
df_event_before_end = working_df.iloc[index_before_End].copy()
df_event_before_end.head()

,org:resource,org:role,remaining_time,id,concept:name,time:timestamp,case:Permit travel permit number,case:DeclarationNumber,case:Amount,case:RequestedAmount,...,case:id,case:Permit ID,case:Permit id,case:BudgetNumber,case:Permit ActivityNumber,case:AdjustedAmount,case:concept:name,trip_duration,remaining_time_since_end_trip,time_submission
0,STAFF MEMBER,EMPLOYEE,4577.514722,rv_travel permit 76455_6,Start trip,2016-10-04 22:00:00+00:00,travel permit number 76456,declaration number 76458,39.664561,39.664561,...,declaration 76457,travel permit 76455,travel permit 76455,budget 144133,activity 46005,39.664561,declaration 76457,NaN,NaN,NaN
1,STAFF MEMBER,EMPLOYEE,4577.514722,rv_travel permit 76455_7,End trip,2016-10-04 22:00:00+00:00,travel permit number 76456,declaration number 76458,39.664561,39.664561,...,declaration 76457,travel permit 76455,travel permit 76455,budget 144133,activity 46005,39.664561,declaration 76457,0.0,4577.514722,NaN
8,STAFF MEMBER,EMPLOYEE,2705.517222,rv_travel permit 76665_6,Start trip,2016-11-20 23:00:00+00:00,travel permit number 76666,declaration number 76668,346.544903,346.544903,...,declaration 76667,travel permit 76665,travel permit 76665,budget 144054,UNKNOWN,346.544903,declaration 76667,NaN,NaN,NaN
9,STAFF MEMBER,EMPLOYEE,1961.517222,rv_travel permit 76665_7,End trip,2016-12-21 23:00:00+00:00,travel permit number 76666,declaration number 76668,346.544903,346.544903,...,declaration 76667,travel permit 76665,travel permit 76665,budget 144054,UNKNOWN,346.544903,declaration 76667,744.0,1961.517222,NaN
16,STAFF MEMBER,EMPLOYEE,1025.530556,rv_travel permit 73652_6,Start trip,2016-12-07 23:00:00+00:00,travel permit number 73653,declaration number 73655,56.972769,56.972769,...,declaration 73654,travel permit 73652,travel permit 73652,budget 143677,UNKNOWN,56.972769,declaration 73654,NaN,NaN,NaN


## Naive Frequency Based Encoding
Frequency based encoding on all events

ToDo: Encoding only on Events before End Trip

In [93]:
eventCategories = df_event_before_end[eventColumnName].unique()
caseIds = df_event_before_end[caseIdColumnName].unique()

frequency_df = pd.DataFrame(index=caseIds, columns=eventCategories).fillna(0)

for cur_case in caseIds:
    cur_eventCategories = df_event_before_end[df_event_before_end[caseIdColumnName] == cur_case][eventColumnName]
    for e in cur_eventCategories:
        frequency_df.loc[cur_case][e] += 1  

### Manually group columns

In [94]:
permit_rejected_columns = ["Permit REJECTED by PRE_APPROVER", "Permit REJECTED by ADMINISTRATION", "Permit REJECTED by BUDGET OWNER", "Permit REJECTED by SUPERVISOR", "Permit REJECTED by EMPLOYEE", "Permit REJECTED by MISSING"]
permit_approved_columns = ["Permit APPROVED by ADMINISTRATION", "Permit APPROVED by SUPERVISOR", "Permit APPROVED by PRE_APPROVER", "Permit APPROVED by BUDGET OWNER"]
permit_final_approved_columns = ["Permit FINAL_APPROVED by DIRECTOR", "Permit FINAL_APPROVED by SUPERVISOR"]

declaration_rejected_columns = ["Declaration REJECTED by MISSING", "Declaration REJECTED by PRE_APPROVER", "Declaration REJECTED by SUPERVISOR", "Declaration REJECTED by ADMINISTRATION", "Declaration REJECTED by EMPLOYEE", "Declaration REJECTED by BUDGET OWNER"]
declaration_approved_columns = ["Declaration APPROVED by PRE_APPROVER", "Declaration APPROVED by ADMINISTRATION", "Declaration APPROVED by BUDGET OWNER"]

### Removal of outlier processes:
* The Process is defined as such, that the decleration is only submitted after the trip has ended.
* In the Data it can be seen that this is not always the case
* For our prediction task we want to focus on tasks that 

In [95]:
declaration_columns = declaration_rejected_columns + declaration_approved_columns
frequency_df.drop(columns=declaration_columns)

,Start trip,End trip,Permit SUBMITTED by EMPLOYEE,Permit APPROVED by SUPERVISOR,Permit FINAL_APPROVED by DIRECTOR,Permit FINAL_APPROVED by SUPERVISOR,Permit APPROVED by PRE_APPROVER,Permit REJECTED by SUPERVISOR,Permit REJECTED by EMPLOYEE,Permit REJECTED by MISSING,Declaration SUBMITTED by EMPLOYEE,Declaration FINAL_APPROVED by SUPERVISOR,Request Payment,Permit REJECTED by PRE_APPROVER,Permit APPROVED by ADMINISTRATION,Permit APPROVED by BUDGET OWNER,Permit REJECTED by ADMINISTRATION,Permit REJECTED by BUDGET OWNER
declaration 76457,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
declaration 76667,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
declaration 73654,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
declaration 73582,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
declaration 72590,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
declaration 12674,1,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
declaration 15973,1,1,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0
declaration 12689,1,1,1,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0
declaration 15436,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Create new features grouping similar events together

In [96]:
frequency_df['permit_rejected'] = frequency_df[permit_rejected_columns].sum(axis=1)
frequency_df['permit_approved'] = frequency_df[permit_approved_columns].sum(axis=1)
frequency_df['permit_final_approved'] = frequency_df[permit_final_approved_columns].sum(axis=1)
# frequency_df['declaration_rejected'] = frequency_df[declaration_rejected_columns].sum(axis=1)
# frequency_df['declaration_approved'] = frequency_df[declaration_approved_columns].sum(axis=1)

In [97]:
frequency_df.head()

,Start trip,End trip,Permit SUBMITTED by EMPLOYEE,Permit APPROVED by SUPERVISOR,Permit FINAL_APPROVED by DIRECTOR,Permit FINAL_APPROVED by SUPERVISOR,Permit APPROVED by PRE_APPROVER,Permit REJECTED by SUPERVISOR,Permit REJECTED by EMPLOYEE,Permit REJECTED by MISSING,...,Declaration APPROVED by ADMINISTRATION,Declaration APPROVED by BUDGET OWNER,Declaration REJECTED by ADMINISTRATION,Declaration REJECTED by EMPLOYEE,Permit REJECTED by ADMINISTRATION,Permit REJECTED by BUDGET OWNER,Declaration REJECTED by BUDGET OWNER,permit_rejected,permit_approved,permit_final_approved
declaration 76457,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
declaration 76667,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
declaration 73654,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
declaration 73582,1,1,1,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
declaration 72590,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Add some interresting features to the dataset

We first compute these features in the initial prefix dataset 

In [98]:
df_event_before_end["time_submission"] = df_event_before_end.groupby("case:concept:name")["time_submission"].transform(lambda x: x.fillna(x.mean()))
df_event_before_end["time_submission"] = df_event_before_end.groupby("case:concept:name")["time_submission"].transform(lambda x: x.fillna(0))
df_event_before_end["trip_duration"] = df_event_before_end.groupby("case:concept:name")["trip_duration"].transform(lambda x: x.fillna(x.mean()))
df_event_before_end["remaining_time_since_end_trip"] = df_event_before_end.groupby("case:concept:name")["remaining_time_since_end_trip"].transform(lambda x: x.fillna(x.mean()))

Then we add these new features to the frequency based encoder dataset

In [99]:
time_sub = []
trip_duration = []
remaining_time = []
start_time_event = []
amount = []


for caseId in caseIds:
    cur_trace = df_event_before_end.loc[df_event_before_end[caseIdColumnName] == caseId]
    time_sub.append(cur_trace["time_submission"].iloc[0])
    trip_duration.append(cur_trace["trip_duration"].iloc[0])
    remaining_time.append(cur_trace["remaining_time_since_end_trip"].iloc[0])
    start_time_event.append(pd.Timestamp(cur_trace["time:timestamp"].iloc[0]))
    amount.append(cur_trace["case:Amount"].iloc[0])

In [100]:
frequency_df["start_time_trace"] = start_time_event
frequency_df["remaining_time_since_end_trip"] = remaining_time
frequency_df["trip_duration"] = trip_duration
frequency_df["time_submission"] = time_sub
frequency_df["amount"] = amount

We sort the dataset by the start time of each traces (useful to split the data into a training and a test set)

In [101]:
frequency_df.sort_values(by='start_time_trace')

,Start trip,End trip,Permit SUBMITTED by EMPLOYEE,Permit APPROVED by SUPERVISOR,Permit FINAL_APPROVED by DIRECTOR,Permit FINAL_APPROVED by SUPERVISOR,Permit APPROVED by PRE_APPROVER,Permit REJECTED by SUPERVISOR,Permit REJECTED by EMPLOYEE,Permit REJECTED by MISSING,...,Permit REJECTED by BUDGET OWNER,Declaration REJECTED by BUDGET OWNER,permit_rejected,permit_approved,permit_final_approved,start_time_trace,remaining_time_since_end_trip,trip_duration,time_submission,amount
declaration 76457,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2016-10-04 22:00:00+00:00,4577.514722,0.0,0.000000,39.664561
declaration 76667,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2016-11-20 23:00:00+00:00,1961.517222,744.0,0.000000,346.544903
declaration 73654,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2016-12-07 23:00:00+00:00,1025.530556,0.0,0.000000,56.972769
declaration 73582,1,1,1,1,1,0,0,0,0,0,...,0,0,0,1,1,2016-12-31 23:00:00+00:00,617.518889,8736.0,0.000000,199.546260
declaration 72590,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2017-01-08 23:00:00+00:00,809.538333,48.0,0.000000,14.569044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
declaration 12674,1,1,1,0,0,0,0,0,0,0,...,0,0,0,1,0,2018-12-07 12:00:16+00:00,1361.522500,24.0,34.995556,43.782713
declaration 15973,1,1,1,0,0,1,0,0,0,0,...,0,0,0,2,1,2018-12-07 21:32:45+00:00,1001.528889,48.0,97.454167,925.091226
declaration 12689,1,1,1,0,0,1,0,0,0,0,...,0,0,0,2,1,2018-12-09 16:41:05+00:00,419.987778,24.0,150.315278,297.421020
declaration 15436,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,2018-12-09 23:00:00+00:00,809.522222,48.0,0.000000,317.596586


## Split the dataset into a training set and a test set

We chose to not consider the overlapping of the different traces and we take the first 80% of the traces according to their starting date.

In [102]:
n = len(start_time_event)
train_set_size = int(n*0.8)

x_train = frequency_df.iloc[:train_set_size, :].drop(["remaining_time_since_end_trip","start_time_trace"], axis=1)
y_train = remaining_time[:train_set_size]
x_test = frequency_df.iloc[train_set_size:, :].drop(["remaining_time_since_end_trip","start_time_trace"], axis=1)
y_test = remaining_time[train_set_size:]

## Training of the model

Now that the data are cleaned and preprocessed, we can start to train our model.

In [103]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=1000,random_state=42)
rf.fit(x_train, y_train)

RandomForestRegressor(n_estimators=1000, random_state=42)

**Evaluate the model**

In [104]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
#Use the random forest model to predict the outcome (remaining time since the trip ended)
predictions = rf.predict(x_test)


#Print the mean absolute error (MAE) and Mean Square Error (MSE)
print('Mean Absolute Error: ', round(mean_absolute_error(y_test, predictions), 2), 'hours.')

print('Mean Square Error: ', round(mean_squared_error(y_test, predictions), 2), ' hours.')

Mean Absolute Error:  420.48 hours.
Mean Square Error:  365082.53  hours.


Try to reduce the number of useless variables and just take the most important one by measuring their importance

In [105]:
feature_list = x_train.columns
# Get numerical feature importances
importances = list(rf.feature_importances_)# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]

Variable: amount               Importance: 0.45
Variable: time_submission      Importance: 0.3
Variable: trip_duration        Importance: 0.14
Variable: permit_approved      Importance: 0.03
Variable: Permit APPROVED by ADMINISTRATION Importance: 0.02
Variable: Permit SUBMITTED by EMPLOYEE Importance: 0.01
Variable: Permit FINAL_APPROVED by SUPERVISOR Importance: 0.01
Variable: Permit APPROVED by PRE_APPROVER Importance: 0.01
Variable: Declaration SUBMITTED by EMPLOYEE Importance: 0.01
Variable: Permit APPROVED by BUDGET OWNER Importance: 0.01
Variable: permit_final_approved Importance: 0.01
Variable: Start trip           Importance: 0.0
Variable: End trip             Importance: 0.0
Variable: Permit APPROVED by SUPERVISOR Importance: 0.0
Variable: Permit FINAL_APPROVED by DIRECTOR Importance: 0.0
Variable: Permit REJECTED by SUPERVISOR Importance: 0.0
Variable: Permit REJECTED by EMPLOYEE Importance: 0.0
Variable: Permit REJECTED by MISSING Importance: 0.0
Variable: Declaration APPROV

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [132]:
rf_most_important = RandomForestRegressor(n_estimators=25, random_state=42)

important_indices = ['amount', 'time_submission', 'trip_duration', "permit_approved"]
train_important = x_train.loc[:, important_indices]
test_important = x_test.loc[:, important_indices]

rf_most_important.fit(train_important, y_train)

predictions2 = rf_most_important.predict(test_important)

print('Mean Absolute Error: ', round(mean_absolute_error(y_test, predictions2), 2), 'hours.')

print('Mean Square Error: ', round(mean_squared_error(y_test, predictions2), 2), ' hours.')



Mean Absolute Error:  436.87 hours.
Mean Square Error:  410508.92  hours.


**Try to find the best hyperparameters with randomized grid search**

In [133]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(5, 200, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(1, 25, num = 9)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 30, 60]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 7, 10, 15]
# Method of selecting samples for training each tree
bootstrap = [True, False]# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [134]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV


rf_hp = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator=rf_hp, param_distributions=random_grid, n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)
rf_random.fit(x_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


/Users/hannes/opt/anaconda3/envs/APM/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/hannes/opt/anaconda3/envs/APM/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/hannes/opt/anaconda3/envs/APM/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remov

/Users/hannes/opt/anaconda3/envs/APM/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/hannes/opt/anaconda3/envs/APM/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/hannes/opt/anaconda3/envs/APM/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remov

/Users/hannes/opt/anaconda3/envs/APM/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/hannes/opt/anaconda3/envs/APM/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/hannes/opt/anaconda3/envs/APM/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remov

/Users/hannes/opt/anaconda3/envs/APM/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/hannes/opt/anaconda3/envs/APM/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/hannes/opt/anaconda3/envs/APM/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remov

/Users/hannes/opt/anaconda3/envs/APM/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/hannes/opt/anaconda3/envs/APM/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/hannes/opt/anaconda3/envs/APM/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remov

/Users/hannes/opt/anaconda3/envs/APM/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/hannes/opt/anaconda3/envs/APM/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/hannes/opt/anaconda3/envs/APM/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remov

/Users/hannes/opt/anaconda3/envs/APM/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/hannes/opt/anaconda3/envs/APM/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Users/hannes/opt/anaconda3/envs/APM/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remov

RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [1, 4, 7, 10, 13, 16, 19,
                                                      22, 25, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4, 7, 10,
                                                             15],
                                        'min_samples_split': [2, 5, 10, 15, 30,
                                                              60],
                                        'n_estimators': [5, 26, 48, 70, 91, 113,
                                                         135, 156, 178, 200]},
                   random_state=42, verbose=2)

In [135]:
rf_random.best_params_

{'n_estimators': 156,
 'min_samples_split': 2,
 'min_samples_leaf': 10,
 'max_features': 'sqrt',
 'max_depth': 7,
 'bootstrap': True}

In [136]:
len(feature_list)

33

Grid Search with cross validation

In [138]:
param_grid_old = {
    'bootstrap': [True],
    'max_depth': [10,15,20],
    'max_features': [5,6],
    'min_samples_leaf': [8, 10, 12],
    'min_samples_split': [1,2,3],
    'n_estimators': [100, 200, 300, 1000]
}

param_grid = {
    'bootstrap': [True],
    'max_depth': [5,7,9],
    'max_features': ["sqrt"],
    'min_samples_leaf': [8, 10, 12],
    'min_samples_split': [2,4,6],
    'n_estimators': [50, 100, 150, 200]
}

rf_opt = RandomForestRegressor()

grid_search = GridSearchCV(estimator=rf, param_grid= param_grid, cv=3, n_jobs=-1, verbose=2)

grid_search.fit(x_train, y_train)

Fitting 3 folds for each of 108 candidates, totalling 324 fits


GridSearchCV(cv=3,
             estimator=RandomForestRegressor(n_estimators=1000,
                                             random_state=42),
             n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [5, 7, 9],
                         'max_features': ['sqrt'],
                         'min_samples_leaf': [8, 10, 12],
                         'min_samples_split': [2, 4, 6],
                         'n_estimators': [50, 100, 150, 200]},
             verbose=2)

In [139]:
best_grid = grid_search.best_estimator_

predictions_opt = grid_search.predict(x_test)

print('Mean Absolute Error: ', round(mean_absolute_error(y_test, predictions_opt), 2), 'hours.')

print('Mean Square Error: ', round(mean_squared_error(y_test, predictions_opt), 2), ' hours.')

Mean Absolute Error:  384.96 hours.
Mean Square Error:  258475.66  hours.


## Comparing the Model with a Naive Predictor
* Compare MSE and MAE with a model that always predicts the average time it takes (based on average time of the training set)

In [140]:
predictions_naive = [np.average(y_train)] * len(y_test)

print('Mean Absolute Error: ', round(mean_absolute_error(y_test, predictions_naive), 2), 'hours.')

print('Mean Square Error: ', round(mean_squared_error(y_test, predictions_naive), 2), ' hours.')

Mean Absolute Error:  386.17 hours.
Mean Square Error:  237099.3  hours.
[CV] END bootstrap=True, max_depth=22, max_features=auto, min_samples_leaf=15, min_samples_split=60, n_estimators=5; total time=   0.1s
[CV] END bootstrap=True, max_depth=22, max_features=auto, min_samples_leaf=4, min_samples_split=15, n_estimators=26; total time=   0.2s
[CV] END bootstrap=False, max_depth=16, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=91; total time=   0.2s
[CV] END bootstrap=True, max_depth=25, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=113; total time=   0.3s
[CV] END bootstrap=True, max_depth=22, max_features=sqrt, min_samples_leaf=4, min_samples_split=60, n_estimators=178; total time=   0.3s
[CV] END bootstrap=False, max_depth=16, max_features=sqrt, min_samples_leaf=7, min_samples_split=10, n_estimators=70; total time=   0.2s
[CV] END bootstrap=False, max_depth=16, max_features=sqrt, min_samples_leaf=7, min_samples_split=10, n_estim

[CV] END bootstrap=False, max_depth=1, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=5; total time=   0.0s
[CV] END bootstrap=True, max_depth=22, max_features=auto, min_samples_leaf=15, min_samples_split=60, n_estimators=5; total time=   0.0s
[CV] END bootstrap=False, max_depth=19, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=91; total time=   0.3s
[CV] END bootstrap=False, max_depth=16, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=91; total time=   0.2s
[CV] END bootstrap=True, max_depth=1, max_features=sqrt, min_samples_leaf=2, min_samples_split=30, n_estimators=135; total time=   0.1s
[CV] END bootstrap=True, max_depth=19, max_features=auto, min_samples_leaf=2, min_samples_split=30, n_estimators=135; total time=   0.9s
[CV] END bootstrap=False, max_depth=4, max_features=sqrt, min_samples_leaf=1, min_samples_split=30, n_estimators=48; total time=   0.1s
[CV] END bootstrap=True, max_depth=4, max_featur

[CV] END bootstrap=True, max_depth=4, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=5; total time=   0.0s
[CV] END bootstrap=False, max_depth=25, max_features=sqrt, min_samples_leaf=2, min_samples_split=15, n_estimators=178; total time=   0.6s
[CV] END bootstrap=False, max_depth=25, max_features=sqrt, min_samples_leaf=7, min_samples_split=10, n_estimators=135; total time=   0.3s
[CV] END bootstrap=False, max_depth=19, max_features=auto, min_samples_leaf=2, min_samples_split=60, n_estimators=135; total time=   1.2s
[CV] END bootstrap=False, max_depth=13, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=70; total time=   0.2s
[CV] END bootstrap=True, max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=30, n_estimators=200; total time=   0.4s
[CV] END bootstrap=True, max_depth=22, max_features=auto, min_samples_leaf=10, min_samples_split=15, n_estimators=26; total time=   0.2s
[CV] END bootstrap=True, max_depth=22, 

[CV] END bootstrap=False, max_depth=1, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=5; total time=   0.0s
[CV] END bootstrap=False, max_depth=25, max_features=sqrt, min_samples_leaf=2, min_samples_split=15, n_estimators=178; total time=   0.5s
[CV] END bootstrap=True, max_depth=25, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=113; total time=   0.2s
[CV] END bootstrap=True, max_depth=22, max_features=sqrt, min_samples_leaf=4, min_samples_split=60, n_estimators=178; total time=   0.3s
[CV] END bootstrap=False, max_depth=19, max_features=auto, min_samples_leaf=2, min_samples_split=60, n_estimators=135; total time=   1.2s
[CV] END bootstrap=False, max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=2, n_estimators=70; total time=   0.7s
[CV] END bootstrap=True, max_depth=25, max_features=sqrt, min_samples_leaf=15, min_samples_split=2, n_estimators=5; total time=   0.0s
[CV] END bootstrap=True, max_depth=25, max_

[CV] END bootstrap=False, max_depth=1, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=5; total time=   0.0s
[CV] END bootstrap=False, max_depth=19, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=91; total time=   0.3s
[CV] END bootstrap=True, max_depth=22, max_features=sqrt, min_samples_leaf=15, min_samples_split=15, n_estimators=91; total time=   0.1s
[CV] END bootstrap=True, max_depth=25, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=113; total time=   0.2s
[CV] END bootstrap=True, max_depth=19, max_features=auto, min_samples_leaf=2, min_samples_split=30, n_estimators=135; total time=   0.8s
[CV] END bootstrap=False, max_depth=16, max_features=auto, min_samples_leaf=15, min_samples_split=5, n_estimators=26; total time=   0.2s
[CV] END bootstrap=True, max_depth=4, max_features=auto, min_samples_leaf=1, min_samples_split=30, n_estimators=200; total time=   0.6s
[CV] END bootstrap=True, max_depth=None, max_